<a href="https://colab.research.google.com/github/ronniewillaert/Biofabrication-Exercises/blob/main/Chapter3_Python_Exercises/Chapter3_Python_Exercise5_MultiOrgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 3 - Exercise 5: Multi-Organ System Integration
## Biofabrication Course - VU Brussels

This notebook covers:
- Body-on-a-chip design principles
- Physiological organ scaling strategies
- Drug ADME modeling (Absorption, Distribution, Metabolism, Excretion)
- Inter-organ communication
- Network topology design

KEY LEARNING FEATURES:
- Design physiologically-scaled organ systems
- Calculate flow distribution networks
- Model drug distribution and metabolism
- Simulate multi-organ drug responses
- Optimize organ connection strategies

In [ ]:
# Chapter 3 - Exercise 5: Multi-Organ System Integration
# Biofabrication Course - VU Brussels
# Interactive Python Exercise

"""
WHAT'S SPECIAL ABOUT EXERCISE 5:

This notebook covers:
• Body-on-a-chip design principles connecting multiple organ modules
• Physiologically-based pharmacokinetic (PBPK) modeling for drug distribution
• Organ scaling ratios to match human physiology (allometric scaling)
• Flow distribution networks ensuring each organ receives appropriate perfusion
• Drug metabolism cascade from absorption through liver to kidney excretion
• Inter-organ communication through secreted factors and metabolites
• Time-course simulation of drug exposure across connected organs
• Sensor integration strategies for real-time multi-organ monitoring

KEY LEARNING FEATURES:
✓ Design physiologically-relevant organ scaling for body-on-chip systems
✓ Calculate flow distribution to match human organ blood flow ratios
✓ Model drug absorption, distribution, metabolism, and excretion (ADME)
✓ Predict drug efficacy and toxicity using multi-organ integration
✓ Optimize organ connection topology (series vs parallel configurations)
✓ Balance throughput with physiological accuracy
✓ Simulate patient-specific responses using personalized parameters
"""

import sys
!{sys.executable} -m pip install matplotlib numpy pandas seaborn plotly -q

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.patches import Rectangle, FancyBboxPatch, Circle, FancyArrowPatch, Polygon
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("="*80)
print("   CHAPTER 3 - EXERCISE 5: MULTI-ORGAN SYSTEM INTEGRATION")
print("="*80)
print()

# ============================================================================
# SECTION 1: PHYSIOLOGICAL SCALING PRINCIPLES
# ============================================================================

print("\n" + "="*80)
print("SECTION 1: ORGAN SCALING FOR BODY-ON-A-CHIP")
print("="*80)

print("""
🫀 HUMAN ORGAN PHYSIOLOGY (70 kg adult):

Organ mass and blood flow distribution:
   • Liver:     1.5 kg (27% of cardiac output)
   • Kidney:    0.3 kg (22% of cardiac output)
   • Heart:     0.3 kg (4% of cardiac output)
   • Lung:      1.0 kg (100% - receives all blood)
   • Gut:       1.0 kg (15% of cardiac output)
   • Brain:     1.4 kg (13% of cardiac output)

🔬 Scaling for microfluidic systems:
   Two approaches:
   1. FUNCTIONAL SCALING: Based on metabolic activity
   2. MASS SCALING: Proportional to organ mass
""")

# Human organ data
human_organs = {
    'Liver': {
        'mass_kg': 1.5,
        'blood_flow_percent': 27,
        'metabolic_activity': 100,  # Relative scale
        'drug_metabolism': True,
        'color': '#e74c3c'
    },
    'Kidney': {
        'mass_kg': 0.3,
        'blood_flow_percent': 22,
        'metabolic_activity': 80,
        'drug_metabolism': False,
        'color': '#9b59b6'
    },
    'Heart': {
        'mass_kg': 0.3,
        'blood_flow_percent': 4,
        'metabolic_activity': 90,
        'drug_metabolism': False,
        'color': '#e67e22'
    },
    'Lung': {
        'mass_kg': 1.0,
        'blood_flow_percent': 100,
        'metabolic_activity': 40,
        'drug_metabolism': False,
        'color': '#1abc9c'
    },
    'Gut': {
        'mass_kg': 1.0,
        'blood_flow_percent': 15,
        'metabolic_activity': 60,
        'drug_metabolism': True,
        'color': '#f39c12'
    }
}

def calculate_chip_scaling(total_volume_uL=1000, scaling_method='functional'):
    """Calculate chip volumes for each organ based on scaling method"""

    if scaling_method == 'mass':
        # Scale by organ mass
        total_mass = sum([org['mass_kg'] for org in human_organs.values()])
        chip_volumes = {}
        for organ, data in human_organs.items():
            chip_volumes[organ] = (data['mass_kg'] / total_mass) * total_volume_uL

    elif scaling_method == 'functional':
        # Scale by metabolic activity and drug interaction
        scaling_factors = {}
        for organ, data in human_organs.items():
            # Weight metabolic activity higher for drug-metabolizing organs
            factor = data['metabolic_activity']
            if data['drug_metabolism']:
                factor *= 1.5  # Extra weight for drug metabolism
            scaling_factors[organ] = factor

        total_factor = sum(scaling_factors.values())
        chip_volumes = {}
        for organ in human_organs.keys():
            chip_volumes[organ] = (scaling_factors[organ] / total_factor) * total_volume_uL

    elif scaling_method == 'blood_flow':
        # Scale by blood flow percentage
        # Exclude lung (gets 100%) and normalize others
        organs_no_lung = {k: v for k, v in human_organs.items() if k != 'Lung'}
        total_flow = sum([org['blood_flow_percent'] for org in organs_no_lung.values()])

        chip_volumes = {}
        for organ, data in human_organs.items():
            if organ == 'Lung':
                chip_volumes[organ] = total_volume_uL * 0.15  # 15% to lung
            else:
                chip_volumes[organ] = (data['blood_flow_percent'] / total_flow) * total_volume_uL * 0.85

    return chip_volumes

# Compare scaling methods
print("\n📊 CHIP VOLUME ALLOCATION (1000 μL total system):")
print("-" * 80)

scaling_methods = ['mass', 'functional', 'blood_flow']
results_df = pd.DataFrame()

for method in scaling_methods:
    volumes = calculate_chip_scaling(1000, method)
    results_df[method.title()] = volumes

results_df['Human Mass (kg)'] = [human_organs[org]['mass_kg'] for org in results_df.index]
results_df['Blood Flow (%)'] = [human_organs[org]['blood_flow_percent'] for org in results_df.index]

print(results_df.to_string())

# ============================================================================
# SECTION 2: FLOW DISTRIBUTION NETWORK
# ============================================================================

print("\n" + "="*80)
print("SECTION 2: DESIGN FLOW DISTRIBUTION NETWORK")
print("="*80)

print("""
🔀 NETWORK TOPOLOGIES:

1. SERIES CONNECTION: Outlet of organ A feeds inlet of organ B
   • Mimics physiological sequence (gut → liver → systemic)
   • Cumulative effects visible
   • Higher O2/glucose depletion

2. PARALLEL CONNECTION: All organs fed from common reservoir
   • Independent organ control
   • Easier troubleshooting
   • Less physiological

3. HYBRID: Combination (e.g., gut→liver in series, others parallel)
   • Best compromise
   • Physiologically relevant
""")

# Design parameters (STUDENTS CAN MODIFY)
total_flow_rate = 200  # μL/min (MODIFY THIS: 50-500)
scaling_method = 'functional'  # MODIFY THIS: 'mass', 'functional', 'blood_flow'
network_topology = 'hybrid'  # MODIFY THIS: 'series', 'parallel', 'hybrid'

print(f"\n🎨 YOUR NETWORK DESIGN:")
print("-" * 50)
print(f"Total flow rate:      {total_flow_rate} μL/min")
print(f"Scaling method:       {scaling_method}")
print(f"Network topology:     {network_topology}")

# Calculate chip volumes
chip_volumes = calculate_chip_scaling(1000, scaling_method)

# Calculate flow rates for each organ based on blood flow ratios
def distribute_flow(total_flow, topology='parallel'):
    """Distribute flow to organs based on topology"""

    if topology == 'parallel':
        # Each organ gets flow proportional to its blood flow %
        flows = {}
        # Normalize excluding lung
        organs_no_lung = {k: v for k, v in human_organs.items() if k != 'Lung'}
        total_blood_flow = sum([org['blood_flow_percent'] for org in organs_no_lung.values()])

        for organ, data in human_organs.items():
            if organ == 'Lung':
                flows[organ] = total_flow  # All blood passes through lung
            else:
                flows[organ] = (data['blood_flow_percent'] / total_blood_flow) * total_flow

    elif topology == 'series':
        # Same flow through all organs
        flows = {organ: total_flow for organ in human_organs.keys()}

    elif topology == 'hybrid':
        # Gut → Liver (series), others parallel
        flows = {}
        gut_liver_flow = total_flow * 0.35  # Combined 42% (27% liver + 15% gut)
        flows['Gut'] = gut_liver_flow
        flows['Liver'] = gut_liver_flow

        remaining_flow = total_flow * 0.65
        organs_parallel = ['Kidney', 'Heart', 'Lung']
        parallel_blood_flow = sum([human_organs[org]['blood_flow_percent']
                                  for org in organs_parallel])

        for organ in organs_parallel:
            flows[organ] = (human_organs[organ]['blood_flow_percent'] /
                          parallel_blood_flow) * remaining_flow

    return flows

organ_flows = distribute_flow(total_flow_rate, network_topology)

print(f"\n💧 FLOW DISTRIBUTION:")
print("-" * 50)
print(f"{'Organ':<12} {'Volume (μL)':<15} {'Flow (μL/min)':<18} {'Residence (min)'}")
print("-" * 50)

for organ in chip_volumes.keys():
    volume = chip_volumes[organ]
    flow = organ_flows[organ]
    residence = volume / flow if flow > 0 else 0
    print(f"{organ:<12} {volume:>10.1f}     {flow:>12.1f}      {residence:>10.2f}")

# ============================================================================
# SECTION 3: DRUG ADME MODELING
# ============================================================================

print("\n" + "="*80)
print("SECTION 3: DRUG ADME (ABSORPTION, DISTRIBUTION, METABOLISM, EXCRETION)")
print("="*80)

print("""
💊 DRUG JOURNEY THROUGH BODY-ON-CHIP:

1. ABSORPTION: Gut absorbs oral drug
2. FIRST-PASS: Liver metabolizes drug before systemic circulation
3. DISTRIBUTION: Drug reaches target organs
4. METABOLISM: Liver converts drug to metabolites
5. EXCRETION: Kidney filters drug/metabolites

Example: Testing new cancer drug
• Dose: 100 μM in gut
• Liver metabolism: 60% conversion
• Target: Heart (looking for cardiotoxicity)
• Kidney clearance: 30% per pass
""")

def simulate_drug_adme(dose_uM=100, gut_absorption=0.8, liver_metabolism=0.6,
                       kidney_clearance=0.3, simulation_time=60, time_step=1):
    """Simulate drug concentration over time in multi-organ system"""

    time_points = np.arange(0, simulation_time + time_step, time_step)

    # Initialize concentrations
    concentrations = {
        'Gut': np.zeros(len(time_points)),
        'Liver': np.zeros(len(time_points)),
        'Heart': np.zeros(len(time_points)),
        'Kidney': np.zeros(len(time_points)),
        'Systemic': np.zeros(len(time_points))
    }

    # Initial dose in gut
    concentrations['Gut'][0] = dose_uM

    for i in range(1, len(time_points)):
        # Gut absorption (exponential decay)
        concentrations['Gut'][i] = concentrations['Gut'][i-1] * (1 - gut_absorption * time_step / 10)
        absorbed = concentrations['Gut'][i-1] * gut_absorption * time_step / 10

        # First-pass liver metabolism
        entering_liver = absorbed
        concentrations['Liver'][i] = entering_liver * (1 - liver_metabolism)
        metabolized = entering_liver * liver_metabolism

        # Distribution to systemic circulation
        to_systemic = concentrations['Liver'][i]
        concentrations['Systemic'][i] = concentrations['Systemic'][i-1] * 0.95 + to_systemic

        # Distribution to heart (target organ)
        concentrations['Heart'][i] = concentrations['Systemic'][i] * 0.3

        # Kidney clearance
        to_kidney = concentrations['Systemic'][i] * 0.2
        concentrations['Kidney'][i] = to_kidney
        cleared = to_kidney * kidney_clearance

        # Update systemic after clearance
        concentrations['Systemic'][i] -= cleared

    return time_points, concentrations

# Run simulation with default parameters (STUDENTS CAN MODIFY)
dose = 100  # μM (MODIFY THIS)
gut_abs = 0.8  # 80% absorption (MODIFY THIS: 0.1-1.0)
liver_met = 0.6  # 60% metabolism (MODIFY THIS: 0.1-0.9)
kidney_clear = 0.3  # 30% clearance (MODIFY THIS: 0.1-0.9)

print(f"\n🧪 DRUG SIMULATION PARAMETERS:")
print("-" * 50)
print(f"Initial dose (gut):   {dose} μM")
print(f"Gut absorption:       {gut_abs*100:.0f}%")
print(f"Liver metabolism:     {liver_met*100:.0f}%")
print(f"Kidney clearance:     {kidney_clear*100:.0f}%")

time, conc = simulate_drug_adme(dose, gut_abs, liver_met, kidney_clear)

# Find peak concentrations and times
print(f"\n📈 PEAK DRUG CONCENTRATIONS:")
print("-" * 50)

for organ in ['Gut', 'Liver', 'Systemic', 'Heart', 'Kidney']:
    peak_conc = np.max(conc[organ])
    peak_time = time[np.argmax(conc[organ])]
    print(f"{organ:<12} {peak_conc:>8.2f} μM at t = {peak_time:.0f} min")

# Therapeutic window check
therapeutic_min = 10  # μM
therapeutic_max = 50  # μM
toxic_threshold = 80  # μM

heart_peak = np.max(conc['Heart'])
print(f"\n✓ THERAPEUTIC ASSESSMENT:")
print("-" * 50)

if heart_peak < therapeutic_min:
    print(f"⚠️ Sub-therapeutic: Peak {heart_peak:.1f} μM < {therapeutic_min} μM")
    print("   Recommendation: Increase dose or improve gut absorption")
elif heart_peak <= therapeutic_max:
    print(f"✅ Therapeutic: {heart_peak:.1f} μM within {therapeutic_min}-{therapeutic_max} μM")
elif heart_peak <= toxic_threshold:
    print(f"⚠️ Near-toxic: {heart_peak:.1f} μM approaching {toxic_threshold} μM")
    print("   Recommendation: Reduce dose or increase clearance")
else:
    print(f"❌ Toxic: {heart_peak:.1f} μM > {toxic_threshold} μM")
    print("   Recommendation: Reject drug candidate or modify formulation")

# ============================================================================
# SECTION 4: INTER-ORGAN COMMUNICATION
# ============================================================================

print("\n" + "="*80)
print("SECTION 4: INTER-ORGAN SIGNALING")
print("="*80)

print("""
📡 ORGAN CROSSTALK:

Beyond drug metabolism, organs communicate via:
• Cytokines (inflammation signals)
• Hormones (insulin, glucagon)
• Metabolites (lactate, ketones)
• Extracellular vesicles (miRNAs)

Example: Liver-Kidney crosstalk
• Liver produces urea from ammonia
• Kidney filters urea for excretion
• Kidney damage → accumulation in system → liver stress
""")

def simulate_crosstalk(initial_signal=10, production_rate_liver=2,
                       clearance_rate_kidney=0.5, simulation_time=120):
    """Simulate signaling molecule exchange between organs"""

    time_points = np.linspace(0, simulation_time, 500)
    dt = time_points[1] - time_points[0]

    # State variables
    liver_signal = np.zeros(len(time_points))
    systemic_signal = np.zeros(len(time_points))
    kidney_signal = np.zeros(len(time_points))

    liver_signal[0] = initial_signal

    for i in range(1, len(time_points)):
        # Liver produces signal
        produced = production_rate_liver * dt
        liver_signal[i] = liver_signal[i-1] + produced

        # Signal released to systemic circulation
        released = liver_signal[i] * 0.1 * dt
        liver_signal[i] -= released
        systemic_signal[i] = systemic_signal[i-1] + released

        # Kidney takes up and clears signal
        taken_up = systemic_signal[i] * 0.2 * dt
        kidney_signal[i] = kidney_signal[i-1] + taken_up
        systemic_signal[i] -= taken_up

        # Kidney clearance
        cleared = kidney_signal[i] * clearance_rate_kidney * dt
        kidney_signal[i] -= cleared

    return time_points, liver_signal, systemic_signal, kidney_signal

# Run crosstalk simulation
time_ct, liver_sig, sys_sig, kidney_sig = simulate_crosstalk()

print(f"\n📊 STEADY-STATE SIGNALING LEVELS:")
print("-" * 50)
print(f"Liver:    {liver_sig[-1]:.2f} units")
print(f"Systemic: {sys_sig[-1]:.2f} units")
print(f"Kidney:   {kidney_sig[-1]:.2f} units")

# ============================================================================
# SECTION 5: COMPREHENSIVE VISUALIZATION
# ============================================================================

print("\n" + "="*80)
print("SECTION 5: MULTI-ORGAN SYSTEM VISUALIZATION")
print("="*80)

fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Plot 1: Organ scaling comparison
ax1 = fig.add_subplot(gs[0, :])
methods = ['Mass', 'Functional', 'Blood_flow']
x = np.arange(len(human_organs))
width = 0.25

for i, method in enumerate(methods):
    volumes = calculate_chip_scaling(1000, method.lower())
    values = [volumes[org] for org in human_organs.keys()]
    ax1.bar(x + i*width, values, width, label=method, alpha=0.7)

ax1.set_xlabel('Organ', fontsize=11, fontweight='bold')
ax1.set_ylabel('Chip Volume (μL)', fontsize=11, fontweight='bold')
ax1.set_title('Organ Scaling Methods Comparison', fontsize=12, fontweight='bold')
ax1.set_xticks(x + width)
ax1.set_xticklabels(human_organs.keys())
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Plot 2: Drug pharmacokinetics
ax2 = fig.add_subplot(gs[1, :2])
for organ in ['Gut', 'Liver', 'Systemic', 'Heart', 'Kidney']:
    ax2.plot(time, conc[organ], linewidth=2, label=organ,
            color=human_organs.get(organ, {'color': '#34495e'})['color'])

ax2.axhspan(therapeutic_min, therapeutic_max, alpha=0.2, color='green', label='Therapeutic window')
ax2.axhline(toxic_threshold, color='red', linestyle='--', linewidth=2, label='Toxic threshold')

ax2.set_xlabel('Time (minutes)', fontsize=11, fontweight='bold')
ax2.set_ylabel('Drug Concentration (μM)', fontsize=11, fontweight='bold')
ax2.set_title('Drug Pharmacokinetics in Multi-Organ System', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# Plot 3: Flow distribution pie chart
ax3 = fig.add_subplot(gs[1, 2])
flow_values = [organ_flows[org] for org in human_organs.keys()]
colors_pie = [human_organs[org]['color'] for org in human_organs.keys()]
ax3.pie(flow_values, labels=human_organs.keys(), autopct='%1.1f%%',
       colors=colors_pie, startangle=90)
ax3.set_title(f'Flow Distribution\n({network_topology} topology)',
             fontsize=12, fontweight='bold')

# Plot 4: Inter-organ signaling
ax4 = fig.add_subplot(gs[2, :2])
ax4.plot(time_ct, liver_sig, linewidth=2, label='Liver', color='#e74c3c')
ax4.plot(time_ct, sys_sig, linewidth=2, label='Systemic', color='#3498db')
ax4.plot(time_ct, kidney_sig, linewidth=2, label='Kidney', color='#9b59b6')

ax4.set_xlabel('Time (minutes)', fontsize=11, fontweight='bold')
ax4.set_ylabel('Signal Concentration (units)', fontsize=11, fontweight='bold')
ax4.set_title('Liver-Kidney Crosstalk Signaling', fontsize=12, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Plot 5: System schematic
ax5 = fig.add_subplot(gs[2, 2])
ax5.set_xlim(0, 10)
ax5.set_ylim(0, 10)
ax5.axis('off')
ax5.set_title('Network Topology', fontsize=12, fontweight='bold')

# Draw simplified organ network
if network_topology == 'series':
    organs_draw = ['Gut', 'Liver', 'Heart', 'Kidney']
    for i, organ in enumerate(organs_draw):
        y = 8 - i*2
        color = human_organs[organ]['color']
        rect = FancyBboxPatch((2, y-0.4), 6, 0.8, boxstyle="round,pad=0.05",
                             edgecolor='black', facecolor=color, alpha=0.6, linewidth=2)
        ax5.add_patch(rect)
        ax5.text(5, y, organ, ha='center', va='center', fontsize=10, fontweight='bold')

        if i < len(organs_draw) - 1:
            ax5.arrow(5, y-0.5, 0, -0.8, head_width=0.3, head_length=0.2,
                     fc='black', ec='black', linewidth=2)

elif network_topology == 'hybrid':
    # Gut → Liver series
    for i, organ in enumerate(['Gut', 'Liver']):
        x = 2
        y = 8 - i*2
        color = human_organs[organ]['color']
        rect = FancyBboxPatch((x-0.8, y-0.4), 1.6, 0.8, boxstyle="round,pad=0.05",
                             edgecolor='black', facecolor=color, alpha=0.6, linewidth=2)
        ax5.add_patch(rect)
        ax5.text(x, y, organ[:4], ha='center', va='center', fontsize=9, fontweight='bold')

        if i == 0:
            ax5.arrow(x, y-0.5, 0, -0.8, head_width=0.2, head_length=0.2,
                     fc='black', ec='black', linewidth=2)

    # Parallel organs
    parallel_organs = ['Heart', 'Kidney', 'Lung']
    for i, organ in enumerate(parallel_organs):
        x = 5 + i*2
        y = 5
        color = human_organs[organ]['color']
        rect = FancyBboxPatch((x-0.6, y-0.4), 1.2, 0.8, boxstyle="round,pad=0.05",
                             edgecolor='black', facecolor=color, alpha=0.6, linewidth=2)
        ax5.add_patch(rect)
        ax5.text(x, y, organ[:4], ha='center', va='center', fontsize=8, fontweight='bold')

ax5.text(5, 1, f'{network_topology.title()} Configuration',
        ha='center', fontsize=10, style='italic')

plt.tight_layout()
plt.show()

# ============================================================================
# SECTION 6: STUDENT EXPLORATION TASKS
# ============================================================================

print("\n" + "="*80)
print("SECTION 6: HANDS-ON EXPLORATION")
print("="*80)

print("""
🎯 STUDENT TASKS:

Task 1: PERSONALIZED MEDICINE
   Simulate drug response for:
   • Patient A: Normal liver function (metabolism = 60%)
   • Patient B: Liver disease (metabolism = 20%)
   • Patient C: Kidney disease (clearance = 10%)
   How should dosing be adjusted for each patient?

Task 2: DRUG-DRUG INTERACTION
   Two drugs administered:
   • Drug A: Requires liver metabolism to become active
   • Drug B: Inhibits liver enzymes (reduces metabolism 50%)
   What happens when both are given together?

Task 3: OPTIMIZE NETWORK TOPOLOGY
   You're testing a drug that:
   • Must pass through liver first (first-pass effect critical)
   • Target is heart (monitor cardiotoxicity)
   • Kidney clearance determines dosing schedule
   Design the optimal organ connection scheme

Task 4: SCALE-UP TO HUMAN
   Your chip shows promising results. Calculate:
   • How to scale volumes 100× for better sensitivity
   • New flow rates to maintain same residence times
   • Pump and reservoir requirements

Task 5: DISEASE MODEL
   Model diabetes:
   • Gut absorbs glucose continuously
   • Liver should store glucose but is insulin-resistant
   • Kidney experiences glucose toxicity at high levels
   Design the system and predict glucose dynamics

MODIFY THESE PARAMETERS IN SECTIONS 2-3 AND RE-RUN:
   total_flow_rate = 200    # Try: 50-500 μL/min
   scaling_method = 'functional'  # Try: 'mass', 'functional', 'blood_flow'
   network_topology = 'hybrid'  # Try: 'series', 'parallel', 'hybrid'

   dose = 100  # Try: 10-500 μM
   gut_abs = 0.8  # Try: 0.1-1.0
   liver_met = 0.6  # Try: 0.1-0.9
   kidney_clear = 0.3  # Try: 0.1-0.9
""")

# ============================================================================
# SECTION 7: REFLECTION QUESTIONS
# ============================================================================

print("\n" + "="*80)
print("SECTION 7: REFLECTION QUESTIONS")
print("="*80)

print("""
Answer these questions based on your exploration:

1. PHYSIOLOGICAL RELEVANCE:
   Which scaling method (mass, functional, blood flow) is most appropriate for:
   a) Drug toxicity testing?
   b) Studying organ crosstalk?
   c) High-throughput screening?

2. NETWORK DESIGN:
   Compare series vs parallel organ connections:
   What are advantages and disadvantages of each?
   When would you use each topology?

3. DRUG DEVELOPMENT:
   A new drug shows 70% liver metabolism. Is this:
   - Good (reduces systemic exposure)?
   - Bad (requires higher doses)?
   How does first-pass effect impact drug design?

4. PERSONALIZED MEDICINE:
   How would you use organ-chips to predict patient-specific drug response?
   What patient parameters would you need to measure?

5. FUTURE DIRECTIONS:
   Current chips use immortalized cell lines.
   How would using patient-derived iPSCs change:
   - Predictive accuracy?
   - Cost and complexity?
   - Regulatory acceptance?

6. INTEGRATION CHALLENGES:
   List 3 major technical challenges for multi-organ chips:
   How would you address each challenge?

Write your answers here:
__________________________________________________________________________
__________________________________________________________________________
__________________________________________________________________________
""")

print("\n" + "="*80)
print("🎉 EXERCISE 5 COMPLETE!")
print("="*80)
print("=" * 80)
print("🎊 CONGRATULATIONS! ALL CHAPTER 3 EXERCISES COMPLETED! 🎊")
print("=" * 80)
print()
print("You have mastered:")
print("✓ Exercise 1: Microfluidic Channel Flow - Reynolds number, shear stress")
print("✓ Exercise 2: Contact Angle & Surface Treatment - Wettability control")
print("✓ Exercise 3: Photolithography - Resolution limits, soft lithography")
print("✓ Exercise 4: Organ Perfusion - Nutrient delivery, cell viability")
print("✓ Exercise 5: Multi-Organ Integration - ADME modeling, system design")
print()
print("🎓 KEY SKILLS ACQUIRED:")
print("  • Microfluidic system design and optimization")
print("  • Surface modification for cell control")
print("  • Microfabrication method selection")
print("  • Perfusion system optimization")
print("  • Multi-organ system integration")
print("  • Pharmacokinetic modeling")
print()
print("📚 Ready to apply these concepts to real organ-on-chip projects!")
print("=" * 80)